<a href="https://colab.research.google.com/github/anthonyibr24/Feature-selection-code/blob/Load_wine/Load_wine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#import
from sklearn.datasets import load_wine
import pandas as pd
import numpy as np

In [ ]:
#load dataset
data=load_wine()
df=pd.DataFrame(data.data,columns=data.feature_names)
df['target']=data.target
df.head()

#scale mean=0 std=1
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
df=df.drop('target',axis=1)
df[data.feature_names]=scaler.fit_transform(df[data.feature_names])
df.describe().round(3)

df['target']=data.target
df.describe().round(3)

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,target
count,178.000,178.000,178.000,178.000,178.000,178.000,178.000,178.000,178.000,178.000,178.000,178.000,178.000,178.000
mean,0.000,0.000,-0.000,-0.000,-0.000,-0.000,0.000,-0.000,-0.000,-0.000,0.000,0.000,-0.000,0.938
std,1.003,1.003,1.003,1.003,1.003,1.003,1.003,1.003,1.003,1.003,1.003,1.003,1.003,0.775
min,-2.434,-1.433,-3.679,-2.671,-2.088,-2.107,-1.696,-1.868,-2.069,-1.634,-2.095,-1.895,-1.493,0.000
25%,-0.788,-0.659,-0.572,-0.689,-0.824,-0.885,-0.828,-0.740,-0.597,-0.795,-0.768,-0.952,-0.785,0.000
50%,0.061,-0.423,-0.024,0.002,-0.122,0.096,0.106,-0.176,-0.063,-0.159,0.033,0.238,-0.234,1.000
75%,0.836,0.670,0.698,0.602,0.510,0.809,0.849,0.610,0.629,0.494,0.713,0.789,0.758,2.000
max,2.260,3.109,3.156,3.155,4.371,2.540,3.063,2.402,3.485,3.435,3.302,1.961,2.971,2.000


In [ ]:
df.shape

(178, 14)

In [ ]:
print(df['target'].value_counts())

target
1    71
0    59
2    48
Name: count, dtype: int64


# Full features

In [ ]:
#Cross validation
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LogisticRegression

lr=LogisticRegression()

cv_results = cross_validate(
    lr,
    df.drop('target', axis=1),
    df['target'],
    cv=4,
    scoring=['accuracy', 'precision_macro', 'recall_macro', 'f1_macro'],
    return_train_score=False
)

from sklearn.model_selection import cross_validate
from sklearn.linear_model import LogisticRegression
import numpy as np  # Needed for mean and std

lr = LogisticRegression()

cv_results = cross_validate(
    lr,
    df.drop('target', axis=1),
    df['target'],
    cv=4,
    scoring=['accuracy', 'precision_macro', 'recall_macro', 'f1_macro'],
    return_train_score=False
)

print("\nAverage Metrics Across 4 Folds:")
print(f"Accuracy:  {np.mean(cv_results['test_accuracy']):.4f} ± {np.std(cv_results['test_accuracy']):.4f}")
print(f"Precision: {np.mean(cv_results['test_precision_macro']):.4f} ± {np.std(cv_results['test_precision_macro']):.4f}")
print(f"Recall:    {np.mean(cv_results['test_recall_macro']):.4f} ± {np.std(cv_results['test_recall_macro']):.4f}")
print(f"F1 Score:  {np.mean(cv_results['test_f1_macro']):.4f} ± {np.std(cv_results['test_f1_macro']):.4f}")



Average Metrics Across 4 Folds:
Accuracy:  0.9777 ± 0.0157
Precision: 0.9773 ± 0.0166
Recall:    0.9806 ± 0.0132
F1 Score:  0.9781 ± 0.0155


# Using 13 features


---


Accuracy:  0.9777


Precision: 0.9773


Recall:    0.9806


F1 Score:  0.9781


---



# Mutual Gain

In [ ]:
#Mutual gain
from sklearn.datasets import load_wine
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

#load dataset
data=load_wine()
df=pd.DataFrame(data.data,columns=data.feature_names)
df['target']=data.target
df.head()

X=df.drop('target',axis=1)
y=df['target']

#Pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('selector', SelectKBest(mutual_info_classif, k=6)),
    ('classifier', LogisticRegression())
])

pipeline.verbose=True

#Cross-validate and calculate all metrics
scores = cross_validate(
    pipeline,
    X, y,
    cv=4,
    scoring=['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']
)

print("\nAverage Scores Across 4 Folds:")
for metric in ['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']:
    mean_score = scores[f'test_{metric}'].mean()
    std_score = scores[f'test_{metric}'].std()
    print(f"{metric.capitalize()}: {mean_score:.4f} ± {std_score:.4f}")

[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=   0.1s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.1s
[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=   0.1s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.0s
[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=   0.1s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.0s
[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=   0.1s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.0s

Average Scores Across 4 Folds:
Accuracy: 0.9778 ± 0.0222
Precision_macro: 0.9819 ± 0.0183
Recall_macro: 0.9796 ± 0.0205
F1_macro: 0.9796 ± 0.0205


In [ ]:
#ANOVA
from sklearn.datasets import load_wine
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline


#load dataset
data=load_wine()
df=pd.DataFrame(data.data,columns=data.feature_names)
df['target']=data.target
df.head()

X=df.drop('target',axis=1)
y=df['target']

#Pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('selector', SelectKBest(score_func=f_classif, k=6)),
    ('classifier', LogisticRegression())
])

pipeline.verbose=True

#Cross-validate and calculate all metrics
scores = cross_validate(
    pipeline,
    X, y,
    cv=4,
    scoring=['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']
)

print("\nAverage Scores Across 4 Folds:")
for metric in ['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']:
    mean_score = scores[f'test_{metric}'].mean()
    std_score = scores[f'test_{metric}'].std()
    print(f"{metric.capitalize()}: {mean_score:.4f} ± {std_score:.4f}")

[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=   0.0s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.2s
[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=   0.0s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.2s
[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=   0.0s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.0s
[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=   0.0s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.0s

Average Scores Across 4 Folds:
Accuracy: 0.9721 ± 0.0184
Precision_macro: 0.9772 ± 0.0152
Recall_macro: 0.9741 ± 0.0170
F1_macro: 0.9743 ± 0.0169


# Wrapper Methods

# Forward

In [ ]:
#FORWARD
from sklearn.datasets import load_wine
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.neighbors import KNeighborsClassifier

#load dataset
data=load_wine()
df=pd.DataFrame(data.data,columns=data.feature_names)
df['target']=data.target
df.head()

X=df.drop('target',axis=1)
y=df['target']

knn=KNeighborsClassifier(5)



In [ ]:
#Pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('selector', SequentialFeatureSelector(knn, n_features_to_select=6, direction='forward', scoring='accuracy')),
    ('classifier', LogisticRegression())
])

pipeline.verbose=True

#Cross-validate and calculate all metrics
scores = cross_validate(
    pipeline,
    X, y,
    cv=4,
    scoring=['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']
)

print("\nAverage Scores Across 4 Folds:")
for metric in ['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']:
    mean_score = scores[f'test_{metric}'].mean()
    std_score = scores[f'test_{metric}'].std()
    print(f"{metric.capitalize()}: {mean_score:.4f} ± {std_score:.4f}")



[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=   2.2s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.1s
[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=   1.7s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.0s
[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=   1.6s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.0s
[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=   1.6s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.0s

Average Scores Across 4 Folds:
Accuracy: 0.9667 ± 0.0333
Precision_macro: 0.9709 ± 0.0291
Recall_macro: 0.9704 ± 0.0297
F1_macro: 0.9688 ± 0.0312


# BACKWARD

In [ ]:
#BACKWARD
from sklearn.datasets import load_wine
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.neighbors import KNeighborsClassifier

#load dataset
data=load_wine()
df=pd.DataFrame(data.data,columns=data.feature_names)
df['target']=data.target
df.head()

X=df.drop('target',axis=1)
y=df['target']

knn=KNeighborsClassifier(5)

In [ ]:
#Pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('selector', SequentialFeatureSelector(knn, n_features_to_select=6, direction='backward', scoring='accuracy')),
    ('classifier', LogisticRegression())
])

pipeline.verbose=True

#Cross-validate and calculate all metrics
scores = cross_validate(
    pipeline,
    X, y,
    cv=4,
    scoring=['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']
)

print("\nAverage Scores Across 4 Folds:")
for metric in ['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']:
    mean_score = scores[f'test_{metric}'].mean()
    std_score = scores[f'test_{metric}'].std()
    print(f"{metric.capitalize()}: {mean_score:.4f} ± {std_score:.4f}")

[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=   1.8s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.0s
[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=   1.8s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.0s
[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=   1.8s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.0s
[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=   1.7s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.0s

Average Scores Across 4 Folds:
Accuracy: 0.9553 ± 0.0270
Precision_macro: 0.9555 ± 0.0297
Recall_macro: 0.9588 ± 0.0257
F1_macro: 0.9562 ± 0.0282


# RFE

In [ ]:
#RFE
from sklearn.datasets import load_wine
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import RFE
from sklearn.svm import SVC

#load dataset
data=load_wine()
df=pd.DataFrame(data.data,columns=data.feature_names)
df['target']=data.target
df.head()

X=df.drop('target',axis=1)
y=df['target']



In [ ]:
#Pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('selector', RFE(estimator=SVC(kernel='linear'), n_features_to_select=6)),
    ('classifier', LogisticRegression())
])
pipeline.verbose=True
#Cross-validate and calculate all metrics
scores = cross_validate(
    pipeline,
    X, y,
    cv=4,
    scoring=['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']
)

print("\nAverage Scores Across 4 Folds:")
for metric in ['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']:
    mean_score = scores[f'test_{metric}'].mean()
    std_score = scores[f'test_{metric}'].std()
    print(f"{metric.capitalize()}: {mean_score:.4f} ± {std_score:.4f}")

[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=   0.0s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.0s
[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=   0.0s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.0s
[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=   0.0s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.0s
[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=   0.0s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.0s

Average Scores Across 4 Folds:
Accuracy: 0.9497 ± 0.0480
Precision_macro: 0.9540 ± 0.0441
Recall_macro: 0.9565 ± 0.0391
F1_macro: 0.9504 ± 0.0487


# Embeded Methods

# Random Forest

In [ ]:
#RF
from sklearn.datasets import load_wine
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel


#load dataset
data=load_wine()
df=pd.DataFrame(data.data,columns=data.feature_names)
df['target']=data.target
df.head()

X=df.drop('target',axis=1)
y=df['target']


In [ ]:
#Pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('selector', SelectFromModel(RandomForestClassifier(random_state=42), max_features=6)),
    ('classifier', LogisticRegression())
])
pipeline.verbose=True

#Cross-validate and calculate all metrics
scores = cross_validate(
    pipeline,
    X, y,
    cv=4,
    scoring=['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']
)

print("\nAverage Scores Across 4 Folds:")
for metric in ['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']:
    mean_score = scores[f'test_{metric}'].mean()
    std_score = scores[f'test_{metric}'].std()
    print(f"{metric.capitalize()}: {mean_score:.4f} ± {std_score:.4f}")

[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=   0.2s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.0s
[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=   0.2s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.0s
[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=   0.2s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.0s
[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=   0.2s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.0s

Average Scores Across 4 Folds:
Accuracy: 0.9722 ± 0.0289
Precision_macro: 0.9750 ± 0.0261
Recall_macro: 0.9750 ± 0.0266
F1_macro: 0.9738 ± 0.0275


# Logistic using L1 penalty

In [ ]:
#LR L1 penalty
from sklearn.datasets import load_wine
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest


#load dataset
data=load_wine()
df=pd.DataFrame(data.data,columns=data.feature_names)
df['target']=data.target
df.head()

X=df.drop('target',axis=1)
y=df['target']


In [ ]:
#Pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('selector', SelectFromModel(LogisticRegression(penalty='l1',solver='saga',C=0.1,max_iter=1000,random_state=42),max_features=6)),
    ('classifier', LogisticRegression())
])
pipeline.verbose=True

#Cross-validate and calculate all metrics
scores = cross_validate(
    pipeline,
    X, y,
    cv=4,
    scoring=['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']
)

print("\nAverage Scores Across 4 Folds:")
for metric in ['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']:
    mean_score = scores[f'test_{metric}'].mean()
    std_score = scores[f'test_{metric}'].std()
    print(f"{metric.capitalize()}: {mean_score:.4f} ± {std_score:.4f}")

[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=   0.0s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.0s
[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=   0.0s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.0s
[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=   0.0s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.0s
[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=   0.0s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.0s

Average Scores Across 4 Folds:
Accuracy: 0.9556 ± 0.0471
Precision_macro: 0.9590 ± 0.0436
Recall_macro: 0.9611 ± 0.0403
F1_macro: 0.9565 ± 0.0464
